## Adapting functions for 2022 stats

In [2]:
#Imports
import statsapi
import datetime 
import pandas as pd
import json
from functools import reduce
import numpy as np


In [3]:
#Used in the player stats function
def add_dicts(d1, d2):
    d3 = dict(d1)
    d3.update(d2)
    try:
        for i, j in d1.items():

            for x, y in d2.items():

                if (i == x):

                    d3[i]=(j+y)
    except: pass
    return d3

In [4]:
#Function to return player stats - hitting, pitching, and fielding

#Inputs are the id, start date and end date and the season year

def player_stats(id,start_date, end_date, year):

    #Getting player stats - separately because the API switches their order when fetched at the same time 
    hitting_stats = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[hitting],type=[byDateRange],startDate={start_date},endDate={end_date},season={year})"})
    pitching_stats = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[pitching],type=[byDateRange],startDate={start_date},endDate={end_date},season={year})"})
    fielding_stats = statsapi.get("people", {"personIds": id, "hydrate": f"stats(group=[fielding],type=[byDateRange],startDate={start_date},endDate={end_date},season={year})"})

    #dict where I will add all the collected stats
    stat_dict = {}

    #Temporary storage for fielding dicts for all positions
    fielding_list = []

    #List to check if the stats for a fielding position have already been collected - as they are listed more than once
    pos_codes = []

    #Taking last entry of hitting stats, as that is the cumulative for the date range
    try:
        player_hitting_stats = hitting_stats['people'][0]['stats'][0]['splits'][-1]['stat']
        stat_dict['hitting'] = player_hitting_stats
    except: pass
    
    #Taking last entry of pitching stats, as that is the cumulative for the date range
    try:
        player_pitching_stats = pitching_stats['people'][0]['stats'][0]['splits'][-1]['stat']
        stat_dict['pitching'] = player_pitching_stats
    except: pass

    #Summing cumulative fielding stats for each position played (Team changes now accounted for)
    try:
        for i in range(len(fielding_stats['people'][0]['stats'][0]['splits'])):
            if fielding_stats['people'][0]['stats'][0]['splits'][i]['sport']['abbreviation'] == 'All':
                fielding_list.append(fielding_stats['people'][0]['stats'][0]['splits'][i]['stat']) 
                
        all_fielding = {}

        for i in range(len(fielding_list)):
            all_fielding = add_dicts(all_fielding, fielding_list[i])
        stat_dict['fielding'] = all_fielding
        
        
    except:pass
    return stat_dict

#### Setting up a few variables

In [5]:
#Start of 2022 season
season_info = statsapi.get("seasons", {"sportId": 1, "season": 2022})
start_date = season_info['seasons'][0]['regularSeasonStartDate']

In [28]:
home_team = input('Enter the name of the home team')
away_team = input('Enter the name of the away team')

In [29]:
home_info = statsapi.lookup_team(home_team)
away_info = statsapi.lookup_team(away_team)

#### General testing for first iteration of script

In [8]:
#Getting home team hitting stats 
h_params = {'season':2022, 'stats':'season', 'group': 'hitting', 'sportIds':1, 'teamId': home_info[0]['id']}
h_hit_stats = statsapi.get('team_stats', h_params, force=True)

In [9]:
#Getting away team hitting stats 
a_params = {'season':2022, 'stats':'season', 'group': 'hitting', 'sportIds':1, 'teamId': away_info[0]['id']}
a_hit_stats = statsapi.get('team_stats', a_params, force=True)

In [10]:
#Checking runs
h_hit_stats['stats'][0]['splits'][0]['stat']['runs']

457

In [11]:
#Getting home team bullpen pitching 
h_pitch_stats = statsapi.get("team_stats", {"teamId": 112, "season": 2021, "group": "pitching", "stats": "statSplits", "sitCodes": "rp"})
h_pitch_stats

{'copyright': 'Copyright 2022 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'stats': [{'type': {'displayName': 'statSplits'},
   'group': {'displayName': 'pitching'},
   'totalSplits': 1,
   'exemptions': [],
   'splits': [{'season': '2021',
     'stat': {'gamesPlayed': 161,
      'gamesStarted': 0,
      'groundOuts': 598,
      'airOuts': 542,
      'runs': 352,
      'doubles': 105,
      'triples': 5,
      'homeRuns': 89,
      'strikeOuts': 715,
      'baseOnBalls': 312,
      'intentionalWalks': 15,
      'hits': 532,
      'hitByPitch': 55,
      'avg': '.226',
      'atBats': 2354,
      'obp': '.328',
      'slg': '.388',
      'ops': '.716',
      'caughtStealing': 11,
      'stolenBases': 43,
      'stolenBasePercentage': '.796',
      'groundIntoDoublePlay': 49,
      'numberOfPitches': 10907,
      'era': '4.39',
      'inningsPitched': '631.0',
      'outsPitched': 

In [12]:
statsapi.get("team_stats", {"teamId": 112, "season": 2021, "group": "pitching", "stats": "statSplits", "sitCodes": "sp"})

{'copyright': 'Copyright 2022 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'stats': [{'type': {'displayName': 'statSplits'},
   'group': {'displayName': 'pitching'},
   'totalSplits': 1,
   'exemptions': [],
   'splits': [{'season': '2021',
     'stat': {'gamesPlayed': 162,
      'gamesStarted': 162,
      'groundOuts': 877,
      'airOuts': 743,
      'runs': 487,
      'doubles': 178,
      'triples': 9,
      'homeRuns': 146,
      'strikeOuts': 643,
      'baseOnBalls': 284,
      'intentionalWalks': 10,
      'hits': 854,
      'hitByPitch': 43,
      'avg': '.278',
      'atBats': 3077,
      'obp': '.346',
      'slg': '.484',
      'ops': '.830',
      'caughtStealing': 10,
      'stolenBases': 50,
      'stolenBasePercentage': '.833',
      'groundIntoDoublePlay': 80,
      'numberOfPitches': 12970,
      'era': '5.27',
      'inningsPitched': '781.2',
      'outsPitched

In [13]:
statsapi.get("team_stats", {"teamId": 112, "season": 2021, "group": "pitching", "stats": "season"})

{'copyright': 'Copyright 2022 MLB Advanced Media, L.P.  Use of any content on this page acknowledges agreement to the terms posted here http://gdx.mlb.com/components/copyright.txt',
 'stats': [{'type': {'displayName': 'season'},
   'group': {'displayName': 'pitching'},
   'exemptions': [],
   'splits': [{'season': '2021',
     'stat': {'gamesPlayed': 162,
      'gamesStarted': 162,
      'groundOuts': 1475,
      'airOuts': 1285,
      'runs': 839,
      'doubles': 283,
      'triples': 14,
      'homeRuns': 235,
      'strikeOuts': 1358,
      'baseOnBalls': 596,
      'intentionalWalks': 25,
      'hits': 1386,
      'hitByPitch': 98,
      'avg': '.255',
      'atBats': 5431,
      'obp': '.338',
      'slg': '.442',
      'ops': '.780',
      'caughtStealing': 21,
      'stolenBases': 93,
      'stolenBasePercentage': '.816',
      'groundIntoDoublePlay': 129,
      'numberOfPitches': 23877,
      'era': '4.87',
      'inningsPitched': '1412.2',
      'wins': 71,
      'losses': 91

In [14]:
#Getting away team bullpen pitching  
a_pitch_stats = statsapi.get("team_stats", {"teamId": 112, "season": 2022, "group": "pitching", "stats": "statSplits", "sitCodes": "rp"})

#### Getting roster lists for each team

In [30]:
#Getting dictionary of team roster from API
h_roster_dict = statsapi.get("team_roster", {"teamId": home_info[0]['id']})
a_roster_dict = statsapi.get("team_roster", {"teamId": away_info[0]['id']})

In [31]:
#Storing player_ids as a list
h_roster_list = [h_roster_dict['roster'][i]['person']['id'] for i in range(len(h_roster_dict['roster']))]
a_roster_list = [a_roster_dict['roster'][i]['person']['id'] for i in range(len(a_roster_dict['roster']))]


### Functions for Log5 statistic

In [17]:
#Function that returns the outcomes for each game, as a dict

def season_match_outcomes(season_start, season_end):
    dict_list = []
    for i in statsapi.schedule(start_date=season_start, end_date=season_end):
        match_dict = {}
        try:
            match_dict['game_id'] = i['game_id']
            match_dict['win'] = i['winning_team']
            match_dict['loss'] = i['losing_team']
            dict_list.append(match_dict)
        except: pass
    return dict_list

In [18]:
#Function that calcualtes a team's win % up to a specific date

#Takes the team_id as an input, as well as the start and end date of the period

def team_record(team_id, start_date, end_date):
    results = season_match_outcomes(start_date, end_date)
    wins = 0
    losses = 0
    team_name = statsapi.lookup_team(team_id)[0]['name']
    for i in range(len(results)):
        if team_name == 'Cleveland Guardians':
            if (results[i]['win'] == team_name) or (results[i]['win'] =='Cleveland Indians'):
                wins+=1
            elif (results[i]['loss'] == team_name) or (results[i]['loss'] =='Cleveland Indians'):
                losses+=1
        else:
            if results[i]['win'] == team_name:
                wins+=1
            elif results[i]['loss'] == team_name:
                losses+=1
    return round(wins/(wins+losses),3)

In [19]:
def log_5(home_team_id, away_team_id, start_date, end_date):
    home_per = team_record(home_team_id, start_date, end_date)
    away_per = team_record(away_team_id, start_date, end_date)
    log5 = (home_per - (home_per * away_per))/(home_per + away_per - (2*home_per*away_per))
    return log5

### Function for all hit stats for 2022

In [20]:
def all_hit_stats_df(df_home, df_away):

    #End date - date of the game input
    end_date = datetime.datetime.today().strftime('%Y-%m-%d')

    #year is used in the player_stats function, and is the first 4 characters of the date string
    year = 2022

    #Id of home team
    home_id = home_info[0]['id']

    #Id of away team
    away_id = away_info[0]['id']
        
    #Getting start date for rolling 10 day stats - 9 days before the end_date
    rolling_start_date = (datetime.datetime.strptime(end_date, "%Y-%m-%d") + datetime.timedelta(days=-9)).strftime('%Y-%m-%d')

    #List of stats being pulled from API
    hitting_stats = ['hits', 'baseOnBalls', 'atBats', 'hitByPitch','doubles', 'triples', 'homeRuns', 'plateAppearances', 'runs']

    #Calculating log5
    log5 = log_5(home_id, away_id, start_date, end_date)

    #Initializing home stats variables for rolling hitting stats
    h_r_hits = 0
    h_r_baseOnBalls = 0
    h_r_atBats = 0
    h_r_hitByPitch = 0
    h_r_doubles = 0
    h_r_triples = 0
    h_r_homeRuns= 0
    h_r_plateAppearances = 0

    #Initializing away stats variables for rolling hitting stats
    a_r_hits = 0
    a_r_baseOnBalls = 0
    a_r_atBats = 0
    a_r_hitByPitch = 0
    a_r_doubles = 0
    a_r_triples = 0
    a_r_homeRuns= 0
    a_r_plateAppearances = 0

    #Getting stats for the home team
    h_hits= h_hit_stats['stats'][0]['splits'][0]['stat']['hits']
    h_baseOnBalls= h_hit_stats['stats'][0]['splits'][0]['stat']['baseOnBalls']
    h_atBats= h_hit_stats['stats'][0]['splits'][0]['stat']['atBats']
    h_hitByPitch= h_hit_stats['stats'][0]['splits'][0]['stat']['hitByPitch']
    h_doubles= h_hit_stats['stats'][0]['splits'][0]['stat']['doubles']
    h_triples= h_hit_stats['stats'][0]['splits'][0]['stat']['triples']
    h_homeRuns= h_hit_stats['stats'][0]['splits'][0]['stat']['homeRuns']
    h_plateAppearances= h_hit_stats['stats'][0]['splits'][0]['stat']['plateAppearances']
    h_runs= h_hit_stats['stats'][0]['splits'][0]['stat']['runs']

    #Getting rolling 10 day stats 
    for j in h_roster_list:
        try:
            #Storing all rolling stats as a list
            stat_list_10 = [player_stats(j, rolling_start_date, end_date, year)['hitting'][i] for i in hitting_stats]
            
            #Adding the home team rolling stats together for the time period
            h_r_hits+= stat_list_10[0]
            h_r_baseOnBalls+= stat_list_10[1]
            h_r_atBats+= stat_list_10[2]
            h_r_hitByPitch+= stat_list_10[3]
            h_r_doubles+= stat_list_10[4]
            h_r_triples+= stat_list_10[5]
            h_r_homeRuns+= stat_list_10[6]
            h_r_plateAppearances+= stat_list_10[7]
        except: pass

    #Getting stats for the away team
    a_hits= a_hit_stats['stats'][0]['splits'][0]['stat']['hits']
    a_baseOnBalls= a_hit_stats['stats'][0]['splits'][0]['stat']['baseOnBalls']
    a_atBats= a_hit_stats['stats'][0]['splits'][0]['stat']['atBats']
    a_hitByPitch= a_hit_stats['stats'][0]['splits'][0]['stat']['hitByPitch']
    a_doubles= a_hit_stats['stats'][0]['splits'][0]['stat']['doubles']
    a_triples= a_hit_stats['stats'][0]['splits'][0]['stat']['triples']
    a_homeRuns= a_hit_stats['stats'][0]['splits'][0]['stat']['homeRuns']
    a_plateAppearances= a_hit_stats['stats'][0]['splits'][0]['stat']['plateAppearances']
    a_runs= a_hit_stats['stats'][0]['splits'][0]['stat']['runs']

    #Getting stats for each player on the away team
    for j in a_roster_list:   
        try:
            #Storing all rolling stats as a list
            stat_list_10 = [player_stats(j, rolling_start_date, end_date, year)['hitting'][i] for i in hitting_stats]

            #Adding the away team rolling stats together for the time period
            a_r_hits+= stat_list_10[0]
            a_r_baseOnBalls+= stat_list_10[1]
            a_r_atBats+= stat_list_10[2]
            a_r_hitByPitch+= stat_list_10[3]
            a_r_doubles+= stat_list_10[4]
            a_r_triples+= stat_list_10[5]
            a_r_homeRuns+= stat_list_10[6]
            a_r_plateAppearances+= stat_list_10[7]
        except: pass
    #appending stats to their respective dfs
    df_home.loc[len(df_home)] = [round(h_hits/h_atBats,3),round((h_hits+h_baseOnBalls+h_hitByPitch)/h_plateAppearances,3),round((h_hits+h_doubles+h_triples*2+h_homeRuns*3)/h_atBats,3),round((h_r_hits+h_r_baseOnBalls+h_r_hitByPitch)/h_r_plateAppearances,3),round((h_r_hits+h_r_doubles+h_r_triples*2+h_r_homeRuns*3)/h_r_atBats,3), log5, h_runs,0]
    df_away.loc[len(df_away)] = [round(a_hits/a_atBats,3),round((a_hits+a_baseOnBalls+a_hitByPitch)/a_plateAppearances,3),round((a_hits+a_doubles+a_triples*2+a_homeRuns*3)/a_atBats,3),round((a_r_hits+a_r_baseOnBalls+a_r_hitByPitch)/a_r_plateAppearances,3),round((a_r_hits+a_r_doubles+a_r_triples*2+a_r_homeRuns*3)/a_r_atBats,3), 0,0,(a_runs*-1)]
    #Returning the difference of home and away team stats
    return df_home.subtract(df_away, fill_value =0)

#### Testing function

In [21]:
#Creating the empty dfs to be populated by all_team_stats

#Name of columns
team_hit_stats = ['avg', 'obp', 'slug', 'obp_10', 'slug_10', 'log_5', 'h_runs', 'a_runs']

#dfs
df_h_h = pd.DataFrame(columns = team_hit_stats)
df_a_h = pd.DataFrame(columns = team_hit_stats)

In [22]:
#Takes 38s
all_hit_stats_df(df_h_h,df_a_h)

,avg,obp,slug,obp_10,slug_10,log_5,h_runs,a_runs
0,0.0,0.0,0.0,0.0,0.0,0.5,457.0,457.0


### Function for all pitching stats for 2022

In [23]:
#Function to calculate pitching stats for home team - away team for a game_id

#Function takes the game_id and home/away team dfs as inputs
#returns a df of home team - away team stats

def all_pitch_stats_df(df_home, df_away):

    #End date - date of the game input
    end_date = datetime.datetime.today().strftime('%Y-%m-%d')

    #year is used in the player_stats function, and is the first 4 characters of the date string
    year = 2022

    #Start date for the 10 day rolling whip
    rolling_start_date = (datetime.datetime.strptime(end_date, "%Y-%m-%d") + datetime.timedelta(days=-9)).strftime('%Y-%m-%d')

    #Id of home team
    home_id = home_info[0]['id']

    #Id of away team
    away_id = away_info[0]['id']
        
    #Getting start date for rolling 10 day stats - 9 days before the end_date
    rolling_start_date = (datetime.datetime.strptime(end_date, "%Y-%m-%d") + datetime.timedelta(days=-9)).strftime('%Y-%m-%d')

    #List of pitching stats used in calculations
    pitching_stats = ['earnedRuns', 'inningsPitched', 'baseOnBalls', 'hits']

    #List of stats for rolling whip average
    whip_stats_list = ['hits', 'inningsPitched', 'baseOnBalls']

    #Getting home starting pitcher
    h_sp = statsapi.schedule(datetime.datetime.today().strftime('%Y-%m-%d'), team = home_info[0]['id'])[0]['home_probable_pitcher']

    #Getting away starting pitcher
    a_sp = statsapi.schedule(datetime.datetime.today().strftime('%Y-%m-%d'), team = home_info[0]['id'])[0]['away_probable_pitcher']


# need to consider if one is not there, then exclude the stats overall for starting pitcher


    #Initializing home team rolling stats variables
    h_r_hits = 0
    h_r_inningsPitched = 0
    h_r_walks = 0

    #Initializing away team rolling stats variables
    a_r_hits = 0
    a_r_inningsPitched = 0
    a_r_walks = 0

    #h_runs is for bullpen earned runs allowed
    h_runs = h_pitch_stats['stats'][0]['splits'][0]['stat']['runs']
    h_inningsPitched = h_pitch_stats['stats'][0]['splits'][0]['stat']['inningsPitched']
    h_walks = h_pitch_stats['stats'][0]['splits'][0]['stat']['baseOnBalls']
    h_hits = h_pitch_stats['stats'][0]['splits'][0]['stat']['hits']
    #This is runs allowed for all players (earned and unearned)
    h_runs_allowed = h_pitch_stats['stats'][0]['splits'][0]['stat']['earnedRuns']

    a_runs = a_pitch_stats['stats'][0]['splits'][0]['stat']['runs']
    a_inningsPitched = a_pitch_stats['stats'][0]['splits'][0]['stat']['inningsPitched']
    a_walks = a_pitch_stats['stats'][0]['splits'][0]['stat']['baseOnBalls']
    a_hits = a_pitch_stats['stats'][0]['splits'][0]['stat']['hits']
    #This is runs allowed for all players (earned and unearned)
    a_runs_allowed = a_pitch_stats['stats'][0]['splits'][0]['stat']['earnedRuns']
    
    #Changing string to float
    h_inningsPitched = float(h_inningsPitched)
    a_inningsPitched = float(a_inningsPitched)

    #Checking if innings pitched ends in a 0.1 or 0.2 and changing this to the proper decimal for 1/3 or 2/3.
    if round(h_inningsPitched - int(h_inningsPitched),1) == 0.2:
        h_inningsPitched = int(h_inningsPitched) + 0.666
    elif round(h_inningsPitched - int(h_inningsPitched),1) == 0.1:
        h_inningsPitched = int(h_inningsPitched) + 0.333

    if round(a_inningsPitched - int(a_inningsPitched),1) == 0.2:
        a_inningsPitched = int(a_inningsPitched) + 0.666
    elif round(a_inningsPitched - int(a_inningsPitched),1) == 0.1:
        a_inningsPitched = int(a_inningsPitched) + 0.333


    #Getting team pitching stats for the home team - may be slightly different that just the bullpen, but only by the starting pitcher
    for j in h_roster_list:
        try:
            #Getting all home team stats for 10 day whip
            stat_list_10 = [player_stats(j, rolling_start_date, end_date, year)['pitching'][i] for i in whip_stats_list]

            #Getting runs allowed  for all possible pitchers
            h_runs_allowed+= player_stats(j, start_date, end_date, year)['pitching']['runs'] 

            #Changing string to float
            stat_list_10[1] = float(stat_list_10[1])

            #Checking if innings pitched ends in a 0.1 or 0.2 and changing this to the proper decimal for 1/3 or 2/3.
            if round(stat_list_10[1] - int(stat_list_10[1]),1) == 0.2:
                stat_list_10[1] = int(stat_list_10[1]) + 0.666
            elif round(stat_list_10[1] - int(stat_list_10[1]),1) == 0.1:
                stat_list_10[1] = int(stat_list_10[1]) + 0.333

            #Summing stats for each team (players) for each game
            h_r_hits+= stat_list_10[0]
            h_r_inningsPitched+= stat_list_10[1]
            h_r_walks+= stat_list_10[2]
        except: pass

    #Getting rolling stats for away team
    for j in a_roster_list:
        try:
            #Getting all away team stats for 10 day whip
            stat_list_10 = [player_stats(j, rolling_start_date, end_date, year)['pitching'][i] for i in whip_stats_list]

            #Getting runs allowed  for all possible pitchers
            a_runs_allowed+= player_stats(j, start_date, end_date, year)['pitching']['runs'] 

            #Changing string to float
            stat_list_10[1] = float(stat_list_10[1])

            #Checking if innings pitched ends in a 0.1 or 0.2 and changing this to the proper decimal for 1/3 or 2/3.
            if round(stat_list_10[1] - int(stat_list_10[1]),1) == 0.2:
                stat_list_10[1] = int(stat_list_10[1]) + 0.666
            elif round(stat_list_10[1] - int(stat_list_10[1]),1) == 0.1:
                stat_list_10[1] = int(stat_list_10[1]) + 0.333

            #Summing stats for each team (players) for each game
            a_r_hits+= stat_list_10[0]
            a_r_inningsPitched+= stat_list_10[1]
            a_r_walks+= stat_list_10[2]
        except: pass
    
    #If either starting pitcher is not announced, setting all stats = 1, so they subtract to 0
    if h_sp == '' or a_sp == '':
        h_sp_runs = 1
        h_sp_inningsPitched = 1
        h_sp_walks = 1
        h_sp_hits = 1
        a_sp_runs = 1
        a_sp_inningsPitched = 1
        a_sp_walks = 1
        a_sp_hits = 1
    else:
        #Getting stats for the home starting pitcher
        h_stat_list = [player_stats(h_sp, start_date, end_date, year)['pitching'][i] for i in pitching_stats]

        #Changing string to float
        h_stat_list[1] = float(h_stat_list[1])

        #Checking if innings pitched ends in a 0.1 or 0.2 and changing this to the proper decimal for 1/3 or 2/3.
        if round(h_stat_list[1] - int(h_stat_list[1]),1) == 0.2:
            h_stat_list[1] = int(h_stat_list[1]) + 0.666
        elif round(h_stat_list[1] - int(h_stat_list[1]),1) == 0.1:
                h_stat_list[1] = int(h_stat_list[1]) + 0.333

        #Summing stats for starting pitcher
        h_sp_runs = h_stat_list[0]
        h_sp_inningsPitched = h_stat_list[1]
        h_sp_walks = h_stat_list[2]
        h_sp_hits = h_stat_list[3]

        #Getting stats for the home starting pitcher
        a_stat_list = [player_stats(a_sp, start_date, end_date, year)['pitching'][i] for i in pitching_stats]

        #Changing string to float
        a_stat_list[1] = float(a_stat_list[1])

        #Checking if innings pitched ends in a 0.1 or 0.2 and changing this to the proper decimal for 1/3 or 2/3.
        if round(a_stat_list[1] - int(a_stat_list[1]),1) == 0.2:
           a_stat_list[1] = int(a_stat_list[1]) + 0.666
        elif round(a_stat_list[1] - int(a_stat_list[1]),1) == 0.1:
            a_stat_list[1] = int(a_stat_list[1]) + 0.333

        a_sp_runs = a_stat_list[0]
        a_sp_inningsPitched = a_stat_list[1]
        a_sp_walks = a_stat_list[2]
        a_sp_hits = a_stat_list[3]

    #appending stats to the df
    df_home.loc[len(df_home)] = [round(h_sp_runs*9/h_sp_inningsPitched,3),round((h_sp_hits+h_sp_walks)/h_sp_inningsPitched,3),round(h_runs*9/h_inningsPitched,3),round((h_hits+h_walks)/h_inningsPitched,3),round((h_r_hits+h_r_walks)/h_r_inningsPitched,3),h_runs_allowed,0]
    df_away.loc[len(df_away)] = [round(a_sp_runs*9/a_sp_inningsPitched,3),round((a_sp_hits+a_sp_walks)/a_sp_inningsPitched,3),round(a_runs*9/a_inningsPitched,3),round((a_hits+a_walks)/a_inningsPitched,3),round((a_r_hits+a_r_walks)/a_r_inningsPitched,3),0,(a_runs_allowed*-1)]
  
    return df_home.subtract(df_away, fill_value =0)

In [24]:
#Creating empty dfs to populate
df_h_p = pd.DataFrame(columns = ['era_sp', 'whip_sp', 'era_bp', 'whip_bp', 'whip_10', 'h_runs_allowed', 'a_runs_allowed'])
df_a_p = pd.DataFrame(columns = ['era_sp', 'whip_sp', 'era_bp', 'whip_bp', 'whip_10', 'h_runs_allowed', 'a_runs_allowed'])

In [25]:
#all_pitch_stats_df(df_h_p, df_a_p)

## Trying it with the other method of fetching data - so it is consistent

#### This is how I got the training data

In [26]:
def all_hit_stats_df(df_home, df_away):

    #End date - date of the game input
    end_date = datetime.datetime.today().strftime('%Y-%m-%d')

    #year is used in the player_stats function, and is the first 4 characters of the date string
    year = 2022

    #Id of home team
    home_id = home_info[0]['id']

    #Id of away team
    away_id = away_info[0]['id']
        
    #Getting start date for rolling 10 day stats - 9 days before the end_date
    rolling_start_date = (datetime.datetime.strptime(end_date, "%Y-%m-%d") + datetime.timedelta(days=-9)).strftime('%Y-%m-%d')

    #List of stats being pulled from API
    hitting_stats = ['hits', 'baseOnBalls', 'atBats', 'hitByPitch','doubles', 'triples', 'homeRuns', 'plateAppearances', 'runs']

    #Calculating log5
    log5 = log_5(home_id, away_id, start_date, end_date)

    #Initializing home stats variables for static hitting stats
    h_hits = 0
    h_baseOnBalls = 0
    h_atBats = 0
    h_hitByPitch = 0
    h_doubles = 0
    h_triples = 0
    h_homeRuns= 0
    h_plateAppearances = 0
    h_runs = 0

    #Initializing home stats variables for rolling hitting stats
    h_r_hits = 0
    h_r_baseOnBalls = 0
    h_r_atBats = 0
    h_r_hitByPitch = 0
    h_r_doubles = 0
    h_r_triples = 0
    h_r_homeRuns= 0
    h_r_plateAppearances = 0

    #Initializing away stats variables for static hitting stats
    a_hits = 0
    a_baseOnBalls = 0
    a_atBats = 0
    a_hitByPitch = 0
    a_doubles = 0
    a_triples = 0
    a_homeRuns= 0
    a_plateAppearances = 0
    a_runs = 0

    #Initializing away stats variables for rolling hitting stats
    a_r_hits = 0
    a_r_baseOnBalls = 0
    a_r_atBats = 0
    a_r_hitByPitch = 0
    a_r_doubles = 0
    a_r_triples = 0
    a_r_homeRuns= 0
    a_r_plateAppearances = 0

    #Getting stats for each player on the home team
    for j in h_roster_list:
        try:
            #Storing all static stats as a list
            stat_list = [player_stats(j, start_date, end_date, year)['hitting'][i] for i in hitting_stats]

            #Adding the home team static stats together for the time period
            h_hits+= stat_list[0]
            h_baseOnBalls+= stat_list[1]
            h_atBats+= stat_list[2]
            h_hitByPitch+= stat_list[3]
            h_doubles+= stat_list[4]
            h_triples+= stat_list[5]
            h_homeRuns+= stat_list[6]
            h_plateAppearances+= stat_list[7]
            h_runs+= stat_list[8]
        except: pass

        try:
            #Storing all rolling stats as a list
            stat_list_10 = [player_stats(j, rolling_start_date, end_date, year)['hitting'][i] for i in hitting_stats]
            
            #Adding the home team rolling stats together for the time period
            h_r_hits+= stat_list_10[0]
            h_r_baseOnBalls+= stat_list_10[1]
            h_r_atBats+= stat_list_10[2]
            h_r_hitByPitch+= stat_list_10[3]
            h_r_doubles+= stat_list_10[4]
            h_r_triples+= stat_list_10[5]
            h_r_homeRuns+= stat_list_10[6]
            h_r_plateAppearances+= stat_list_10[7]
        except: pass

    #Getting stats for each player on the away team
    for j in a_roster_list:
        try:
            #Storing all static stats as a list
            stat_list = [player_stats(j, start_date, end_date, year)['hitting'][i] for i in hitting_stats]

            #Adding the away team stats together for the time period
            a_hits+= stat_list[0]
            a_baseOnBalls+= stat_list[1]
            a_atBats+= stat_list[2]
            a_hitByPitch+= stat_list[3]
            a_doubles+= stat_list[4]
            a_triples+= stat_list[5]
            a_homeRuns+= stat_list[6]
            a_plateAppearances+= stat_list[7]
            a_runs+= stat_list[8]
        except: pass
        
        try:
            #Storing all rolling stats as a list
            stat_list_10 = [player_stats(j, rolling_start_date, end_date, year)['hitting'][i] for i in hitting_stats]

            #Adding the away team rolling stats together for the time period
            a_r_hits+= stat_list_10[0]
            a_r_baseOnBalls+= stat_list_10[1]
            a_r_atBats+= stat_list_10[2]
            a_r_hitByPitch+= stat_list_10[3]
            a_r_doubles+= stat_list_10[4]
            a_r_triples+= stat_list_10[5]
            a_r_homeRuns+= stat_list_10[6]
            a_r_plateAppearances+= stat_list_10[7]
        except: pass


    #appending stats to their respective dfs
    df_home.loc[len(df_home)] = [round(h_hits/h_atBats,3),round((h_hits+h_baseOnBalls+h_hitByPitch)/h_plateAppearances,3),round((h_hits+h_doubles+h_triples*2+h_homeRuns*3)/h_atBats,3),round((h_r_hits+h_r_baseOnBalls+h_r_hitByPitch)/h_r_plateAppearances,3),round((h_r_hits+h_r_doubles+h_r_triples*2+h_r_homeRuns*3)/h_r_atBats,3), log5, h_runs,0]
    df_away.loc[len(df_away)] = [round(a_hits/a_atBats,3),round((a_hits+a_baseOnBalls+a_hitByPitch)/a_plateAppearances,3),round((a_hits+a_doubles+a_triples*2+a_homeRuns*3)/a_atBats,3),round((a_r_hits+a_r_baseOnBalls+a_r_hitByPitch)/a_r_plateAppearances,3),round((a_r_hits+a_r_doubles+a_r_triples*2+a_r_homeRuns*3)/a_r_atBats,3), 0,0,(a_runs*-1)]
    #Returning the difference of home and away team stats
    return df_home.subtract(df_away, fill_value =0)

#### Test

In [27]:
#Creating the empty dfs to be populated by all_team_stats

#Name of columns
team_hit_stats = ['avg', 'obp', 'slug', 'obp_10', 'slug_10', 'log_5', 'h_runs', 'a_runs']

#dfs
df_h_h = pd.DataFrame(columns = team_hit_stats)
df_a_h = pd.DataFrame(columns = team_hit_stats)

In [238]:
#Takes 2 mins
all_hit_stats_df(df_h_h,df_a_h)

,avg,obp,slug,obp_10,slug_10,log_5,h_runs,a_runs
0,-0.017,-0.003,0.001,-0.074,-0.076,0.437779,419.0,434.0


#### Pitching data v2

In [32]:
def all_pitch_stats_df(df_home, df_away):

    #End date - date of the game input
    end_date = datetime.datetime.today().strftime('%Y-%m-%d')

    #year is used in the player_stats function, and is the first 4 characters of the date string
    year = 2022

    #Start date for the 10 day rolling whip
    rolling_start_date = (datetime.datetime.strptime(end_date, "%Y-%m-%d") + datetime.timedelta(days=-9)).strftime('%Y-%m-%d')

    #Id of home team
    home_id = home_info[0]['id']

    #Id of away team
    away_id = away_info[0]['id']

    #List of pitching stats used in calculations
    pitching_stats = ['earnedRuns', 'inningsPitched', 'baseOnBalls', 'hits']

    #List of stats for rolling whip average
    whip_stats_list = ['hits', 'inningsPitched', 'baseOnBalls']

    #Try/except since sometimes a starting is pitching for the first time and has no information stored here.
    try:
        #Getting home starting pitcher
        h_sp = statsapi.lookup_player(statsapi.schedule(datetime.datetime.today().strftime('%Y-%m-%d'), team = home_info[0]['id'])[0]['home_probable_pitcher'])[0]['id']
    except: h_sp = ''
    
    try:
        #Getting away starting pitcher
        a_sp = statsapi.lookup_player(statsapi.schedule(datetime.datetime.today().strftime('%Y-%m-%d'), team = home_info[0]['id'])[0]['away_probable_pitcher'])[0]['id']
    except: a_sp = ''

    #Initializing home team static stats variables
    #h_runs is for bullpen earned runs allowed
    h_runs = 0
    h_inningsPitched = 0
    h_walks = 0
    h_hits = 0
    #This is runs allowed for all players (earned and unearned)
    h_runs_allowed = 0

    #Initializing away team static stats variables
    a_runs = 0
    a_inningsPitched = 0
    a_walks = 0
    a_hits = 0
    #This is runs allowed for all players (earned and unearned)
    a_runs_allowed = 0

    #Initializing home team rolling stats variables
    h_r_hits = 0
    h_r_inningsPitched = 0
    h_r_walks = 0

    #Initializing away team rolling stats variables
    a_r_hits = 0
    a_r_inningsPitched = 0
    a_r_walks = 0

#Checking if starting pitcher is known yet
    if h_sp == '' or a_sp == '':
        h_sp_runs = 1
        h_sp_inningsPitched = 1
        h_sp_walks = 1
        h_sp_hits = 1
        a_sp_runs = 1
        a_sp_inningsPitched = 1
        a_sp_walks = 1
        a_sp_hits = 1
    else:
        #Getting stats for the starting pitcher
        h_stat_list = [player_stats(h_sp, start_date, end_date, year)['pitching'][i] for i in pitching_stats]

        #Changing string to float
        h_stat_list[1] = float(h_stat_list[1])

        #Checking if innings pitched ends in a 0.1 or 0.2 and changing this to the proper decimal for 1/3 or 2/3.
        if round(h_stat_list[1] - int(h_stat_list[1]),1) == 0.2:
            h_stat_list[1] = int(h_stat_list[1]) + 0.666
        elif round(h_stat_list[1] - int(h_stat_list[1]),1) == 0.1:
            h_stat_list[1] = int(h_stat_list[1]) + 0.333

        #Summing stats for starting pitcher
        h_sp_runs = h_stat_list[0]
        h_sp_inningsPitched = h_stat_list[1]
        h_sp_walks = h_stat_list[2]
        h_sp_hits = h_stat_list[3]

        a_stat_list = [player_stats(a_sp, start_date, end_date, year)['pitching'][i] for i in pitching_stats]

        #Changing string to float
        a_stat_list[1] = float(a_stat_list[1])

        #Checking if innings pitched ends in a 0.1 or 0.2 and changing this to the proper decimal for 1/3 or 2/3.
        if round(a_stat_list[1] - int(a_stat_list[1]),1) == 0.2:
            a_stat_list[1] = int(a_stat_list[1]) + 0.666
        elif round(a_stat_list[1] - int(a_stat_list[1]),1) == 0.1:
            a_stat_list[1] = int(a_stat_list[1]) + 0.333

        a_sp_runs = a_stat_list[0]
        a_sp_inningsPitched = a_stat_list[1]
        a_sp_walks = a_stat_list[2]
        a_sp_hits = a_stat_list[3]

    #Getting stats of all players in the bullpen for the home team this game
    for j in h_roster_list:
        try:
            if player_stats(j, start_date, end_date, year)['pitching']['gamesStarted'] <= 8: 
                #Getting all home team stats
                stat_list = [player_stats(j, start_date, end_date, year)['pitching'][i] for i in pitching_stats]
                #Changing string to float
                stat_list[1] = float(stat_list[1])

                #Checking if innings pitched ends in a 0.1 or 0.2 and changing this to the proper decimal for 1/3 or 2/3.
                if round(stat_list[1] - int(stat_list[1]),1) == 0.2:
                    stat_list[1] = int(stat_list[1]) + 0.666
                elif round(stat_list[1] - int(stat_list[1]),1) == 0.1:
                    stat_list[1] = int(stat_list[1]) + 0.333

                #Summing stats for each team (players) for each game
                h_runs+= stat_list[0]
                h_inningsPitched+= stat_list[1]
                h_walks+= stat_list[2]
                h_hits+= stat_list[3]
        except: pass
    for j in h_roster_list:
        try:
            #Getting all home team stats for 10 day whip
            stat_list_10 = [player_stats(j, rolling_start_date, end_date, year)['pitching'][i] for i in whip_stats_list]

            #Getting runs allowed  for all possible pitchers
            h_runs_allowed+= player_stats(j, start_date, end_date, year)['pitching']['runs'] 

            #Changing string to float
            stat_list_10[1] = float(stat_list_10[1])

            #Checking if innings pitched ends in a 0.1 or 0.2 and changing this to the proper decimal for 1/3 or 2/3.
            if round(stat_list_10[1] - int(stat_list_10[1]),1) == 0.2:
                stat_list_10[1] = int(stat_list_10[1]) + 0.666
            elif round(stat_list_10[1] - int(stat_list_10[1]),1) == 0.1:
                stat_list_10[1] = int(stat_list_10[1]) + 0.333

            #Summing stats for each team (players) for each game
            h_r_hits+= stat_list_10[0]
            h_r_inningsPitched+= stat_list_10[1]
            h_r_walks+= stat_list_10[2]

        except: pass

    for j in a_roster_list:
        try:
            if player_stats(j, start_date, end_date, year)['pitching']['gamesStarted'] <= 8: 
                #Getting all away team stats
                stat_list = [player_stats(j, start_date, end_date, year)['pitching'][i] for i in pitching_stats]
                #Changing string to float
                stat_list[1] = float(stat_list[1])

                #Checking if innings pitched ends in a 0.1 or 0.2 and changing this to the proper decimal for 1/3 or 2/3.
                if round(stat_list[1] - int(stat_list[1]),1) == 0.2:
                    stat_list[1] = int(stat_list[1]) + 0.666
                elif round(stat_list[1] - int(stat_list[1]),1) == 0.1:
                    stat_list[1] = int(stat_list[1]) + 0.333

                a_runs+= stat_list[0]
                a_inningsPitched+= stat_list[1]
                a_walks+= stat_list[2]
                a_hits+= stat_list[3]
        except: pass
    for j in a_roster_list:
        try:
            #Getting all away team stats for 10 day whip
            stat_list_10 = [player_stats(j, rolling_start_date, end_date, year)['pitching'][i] for i in whip_stats_list]

            #Getting runs allowed  for all possible pitchers
            a_runs_allowed+= player_stats(j, start_date, end_date, year)['pitching']['runs'] 

            #Changing string to float
            stat_list_10[1] = float(stat_list_10[1])

            #Checking if innings pitched ends in a 0.1 or 0.2 and changing this to the proper decimal for 1/3 or 2/3.
            if round(stat_list_10[1] - int(stat_list_10[1]),1) == 0.2:
                stat_list_10[1] = int(stat_list_10[1]) + 0.666
            elif round(stat_list_10[1] - int(stat_list_10[1]),1) == 0.1:
                stat_list_10[1] = int(stat_list_10[1]) + 0.333

            #Summing stats for each team (players) for each game
            a_r_hits+= stat_list_10[0]
            a_r_inningsPitched+= stat_list_10[1]
            a_r_walks+= stat_list_10[2]
            
        except: pass
    #appending stats to the df
    df_home.loc[len(df_home)] = [round(h_sp_runs*9/h_sp_inningsPitched,3),round((h_sp_hits+h_sp_walks)/h_sp_inningsPitched,3),round(h_runs*9/h_inningsPitched,3),round((h_hits+h_walks)/h_inningsPitched,3),round((h_r_hits+h_r_walks)/h_r_inningsPitched,3),h_runs_allowed,0]
    df_away.loc[len(df_away)] = [round(a_sp_runs*9/a_sp_inningsPitched,3),round((a_sp_hits+a_sp_walks)/a_sp_inningsPitched,3),round(a_runs*9/a_inningsPitched,3),round((a_hits+a_walks)/a_inningsPitched,3),round((a_r_hits+a_r_walks)/a_r_inningsPitched,3),0,(a_runs_allowed*-1)]
  
    return df_home.subtract(df_away, fill_value =0)

## Checking why error ocurred for this pitcher

##### First time pitcher, stats do not exist - no player id

In [465]:
statsapi.schedule(datetime.datetime.today().strftime('%Y-%m-%d'), team = home_info[0]['id'])[0]['away_probable_pitcher']

'Ken Waldichuk'

In [468]:
statsapi.lookup_player('Ken Waldichuk')

[]

In [467]:
statsapi.lookup_player(statsapi.schedule(datetime.datetime.today().strftime('%Y-%m-%d'), team = home_info[0]['id'])[0]['away_probable_pitcher'])[0]['id']

[]

#### Testing function after fix for above

In [ ]:
#Creating empty dfs to populate
df_h_p = pd.DataFrame(columns = ['era_sp', 'whip_sp', 'era_bp', 'whip_bp', 'whip_10', 'h_runs_allowed', 'a_runs_allowed'])
df_a_p = pd.DataFrame(columns = ['era_sp', 'whip_sp', 'era_bp', 'whip_bp', 'whip_10', 'h_runs_allowed', 'a_runs_allowed'])

In [ ]:
all_pitch_stats_df(df_h_p, df_a_p)

## Putting it all together for all_team_stats

In [33]:
#Function that compiles all stats for home team - away team

#Inputs are game id and 4 total empty dfs for home and away hitting and pitching

def all_team_stats(df_h_h, df_a_h, df_h_p, df_a_p):

    #Hitting stats df
    df_h = all_hit_stats_df(df_h_h, df_a_h)
    
    #Pitching stats df
    df_p = all_pitch_stats_df(df_h_p, df_a_p)

    #Calculating PE_diff for home and away teams
    PE_home = (df_h['h_runs']**(1.83))/((df_h['h_runs']**(1.83)+ df_p['h_runs_allowed']**(1.83)))
    PE_away = (df_h['a_runs']**(1.83))/((df_h['a_runs']**(1.83)+ df_p['a_runs_allowed']**(1.83)))
    PE_diff = PE_home - PE_away

    #Dropping runs and runs allowed columns
    df_h.drop(columns = ['h_runs', 'a_runs'], axis=1, inplace=True)
    df_p.drop(columns = ['h_runs_allowed', 'a_runs_allowed'], axis=1, inplace=True)

    #Creating PE column
    df_h['PE'] = PE_diff

    #define list of DataFrames
    dfs = [df_h, df_p]

    #merge all DataFrames into one
    final_df = reduce(lambda left,right: pd.merge(left,right,left_index = True, right_index=True, how='outer'), dfs)
    return final_df

#### Testing all team stats

In [34]:
#Creating the empty dfs to be populated by all_team_stats

#Name of columns
team_hit_stats = ['avg', 'obp', 'slug', 'obp_10', 'slug_10', 'log_5', 'h_runs', 'a_runs']

#dfs
df_h_h = pd.DataFrame(columns = team_hit_stats)
df_a_h = pd.DataFrame(columns = team_hit_stats)
df_h_p = pd.DataFrame(columns = ['era_sp', 'whip_sp', 'era_bp', 'whip_bp', 'whip_10', 'h_runs_allowed', 'a_runs_allowed'])
df_a_p = pd.DataFrame(columns = ['era_sp', 'whip_sp', 'era_bp', 'whip_bp', 'whip_10', 'h_runs_allowed', 'a_runs_allowed'])

In [35]:
#Testing Pirates/Mets on Sept 6 2022
all_team_stats(df_h_h,df_a_h,df_h_p,df_a_p)

,avg,obp,slug,obp_10,slug_10,log_5,PE,era_sp,whip_sp,era_bp,whip_bp,whip_10
0,-0.027,-0.038,-0.05,-0.001,-0.014,0.254829,-0.380213,0.979,0.245,1.752,0.304,0.184


## Tests for a clean script

In [499]:
def test_run():
    #Inputting both teams in the matchup
    home_team = input('Enter the name of the home team')
    away_team = input('Enter the name of the away team')

    #Getting basic team info
    home_info = statsapi.lookup_team(home_team)
    away_info = statsapi.lookup_team(away_team)

    #Getting dictionary of team roster from API
    h_roster_dict = statsapi.get("team_roster", {"teamId": home_info[0]['id']})
    a_roster_dict = statsapi.get("team_roster", {"teamId": away_info[0]['id']})

    #Storing player_ids as a list
    h_roster_list = [h_roster_dict['roster'][i]['person']['id'] for i in range(len(h_roster_dict['roster']))]
    a_roster_list = [a_roster_dict['roster'][i]['person']['id'] for i in range(len(a_roster_dict['roster']))]

    #Creating the empty dfs to be populated by all_team_stats

    #Name of columns
    team_hit_stats = ['avg', 'obp', 'slug', 'obp_10', 'slug_10', 'log_5', 'h_runs', 'a_runs']
    team_pitch_stats = ['era_sp', 'whip_sp', 'era_bp', 'whip_bp', 'whip_10', 'h_runs_allowed', 'a_runs_allowed']

    #dfs
    df_h_h = pd.DataFrame(columns = team_hit_stats)
    df_a_h = pd.DataFrame(columns = team_hit_stats)
    df_h_p = pd.DataFrame(columns = team_pitch_stats)
    df_a_p = pd.DataFrame(columns = team_pitch_stats)

    #Executing all_team_stats function
    df = all_team_stats(df_h_h,df_a_h,df_h_p,df_a_p)
    return df

In [ ]:
test_run()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


df = pd.read_csv('Games_1_100.csv')

#Defining X and y variables
X = df.drop(['game_id','home_win'], axis=1)
y = df['home_win']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)


#Script for Logistical Regression
for name,method in [('RandomForestClassifier', RandomForestClassifier(n_estimators = 10, criterion = 'entropy',random_state=10))]: 
    method.fit(x_train,y_train)
    predict = method.predict(x_test)
    target_names=['loss', 'win']
    print(accuracy_score(y_test, predict))
    print(confusion_matrix(y_test,predict))  
    print(classification_report(y_test,predict,target_names=target_names))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


df = pd.read_csv('Games_1_100.csv')

#Defining X and y variables
X = df.drop(['game_id','home_win'], axis=1)
y = df['home_win']

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

n_estimators = 10
max_depth = 6
max_features = 3 

#Script for Logistical Regression
model = RandomForestClassifier(n_estimators = n_estimators, max_depth = max_depth, max_features = max_features, random_state=10)
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
target_names=['loss', 'win']
print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred,target_names=target_names))

In [ ]:
Xnew = test_run()
# make a prediction
ynew = method.predict(Xnew)
print(Xnew)
print(ynew[0])